In [1]:
import time
import os
import keyring
import cfbd
import duckdb

import numpy as np
import pandas as pd

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///cfb.db

In [2]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = os.environ["CFBD_API_KEY"] #keyring.get_password('system', 'CFBD_API_KEY') #
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

In [4]:
games_api = cfbd.GamesApi(api_config)
con = duckdb.connect('cfb.db')
for year in range(2014,2023):
    season_games = games_api.get_games(year=year,season_type='both')
    season_games_df = pd.DataFrame.from_records([g.to_dict() for g in season_games])
    con.execute("INSERT INTO games SELECT * from season_games_df")
con.commit()


In [ ]:
games_api = cfbd.GamesApi(api_config)
con = duckdb.connect('cfb.db')
for week in range(1,11):
    season_games = games_api.get_games(year=2023, week=week,season_type='both')
    season_games_df = pd.DataFrame.from_records([g.to_dict() for g in season_games])
    con.execute("INSERT INTO games SELECT * from season_games_df")
con.commit()

In [9]:
%%sql 
SELECT *
FROM games
order by start_date desc
LIMIT 10

,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,...,away_conference,away_division,away_points,away_line_scores,away_post_win_prob,away_pregame_elo,away_postgame_elo,excitement_index,highlights,notes
0,401524045,2023,10,regular,2023-11-05T02:30:00.000Z,False,True,False,True,NaN,...,Pac-12,fbs,10.0,"[0, 7, 3, 0]",0.004609,1709.0,1671.0,4.841393,None,None
1,401544152,2023,10,regular,2023-11-05T02:00:00.000Z,False,True,False,True,NaN,...,So. Cal.,iii,7.0,"[0, 0, 7, 0]",NaN,NaN,NaN,NaN,None,None
2,401524046,2023,10,regular,2023-11-05T02:00:00.000Z,False,True,False,True,NaN,...,Pac-12,fbs,26.0,"[7, 7, 6, 6]",0.996662,1786.0,1783.0,4.191301,None,None
3,401550345,2023,10,regular,2023-11-05T02:00:00.000Z,False,True,False,True,NaN,...,So. Cal.,iii,30.0,"[0, 17, 0, 13]",NaN,NaN,NaN,NaN,None,None
4,401550335,2023,10,regular,2023-11-05T02:00:00.000Z,False,True,False,True,NaN,...,So. Cal.,iii,7.0,"[0, 0, 0, 7]",NaN,NaN,NaN,NaN,None,None
5,401532608,2023,10,regular,2023-11-05T02:00:00.000Z,False,True,False,True,NaN,...,Mountain West,fbs,30.0,"[3, 7, 7, 13]",0.617705,1585.0,1582.0,5.577169,None,None
6,401524050,2023,10,regular,2023-11-05T01:00:00.000Z,False,True,False,True,NaN,...,Pac-12,fbs,10.0,"[0, 0, 7, 3]",0.606324,1230.0,1244.0,7.107037,None,None
7,401520378,2023,10,regular,2023-11-05T01:00:00.000Z,False,True,False,True,NaN,...,Conference USA,fbs,21.0,"[0, 0, 14, 7]",0.863661,1505.0,1510.0,4.989515,None,None
8,401550879,2023,10,regular,2023-11-05T01:00:00.000Z,False,True,False,True,NaN,...,Northwest,iii,24.0,"[0, 12, 6, 6]",NaN,NaN,NaN,NaN,None,None
9,401540263,2023,10,regular,2023-11-05T00:00:00.000Z,False,True,False,True,NaN,...,Big Sky,fcs,7.0,"[7, 0, 0, 0]",0.002405,NaN,NaN,4.289741,None,None


In [12]:
%%sql
select distinct
    Season,
    Week,
    season_type
from games
order by season, week, season_type

,season,week,season_type
0,2014,1,postseason
1,2014,1,regular
2,2014,2,regular
3,2014,3,regular
4,2014,4,regular
...,...,...,...
166,2023,6,regular
167,2023,7,regular
168,2023,8,regular
169,2023,9,regular


In [27]:
games_api = cfbd.GamesApi(api_config)
cal_df = pd.DataFrame()
for year in range(2014,2024):
    season_games = games_api.get_calendar(year=year)
    season_games_df = pd.DataFrame.from_records([g.to_dict() for g in season_games])
    cal_df = pd.concat([cal_df,season_games_df])

In [28]:
cal_df.tail()

,season,week,season_type,first_game_start,last_game_start
11,2023,12,regular,2023-11-15T00:00:00.000Z,2023-11-19T03:30:00.000Z
12,2023,13,regular,2023-11-21T05:00:00.000Z,2023-11-26T04:00:00.000Z
13,2023,15,regular,2023-12-09T20:00:00.000Z,2023-12-09T20:00:00.000Z
14,2023,11,postseason,2023-11-11T17:00:00.000Z,2023-11-11T21:00:00.000Z
15,2023,12,postseason,2023-11-18T18:00:00.000Z,2023-11-18T18:00:00.000Z


In [31]:
con = duckdb.connect('cfb.db')
#con.execute("DROP TABLE calendar")
con.execute("CREATE TABLE calendar AS SELECT * FROM cal_df")

In [3]:
%%sql cal_df <<
select *
from calendar
WHERE first_game_start > '2023-10-08T04:00:00.000Z'

In [4]:
#con = duckdb.connect('cfb.db')
#cal_df = con.table('calendar').to_df()

cal_df

,season,week,season_type,first_game_start,last_game_start
0,2023,7,regular,2023-10-10T23:00:00.000Z,2023-10-15T03:00:00.000Z
1,2023,8,regular,2023-10-17T23:00:00.000Z,2023-10-22T02:30:00.000Z
2,2023,9,regular,2023-10-24T23:00:00.000Z,2023-10-30T19:00:00.000Z
3,2023,10,regular,2023-10-31T23:00:00.000Z,2023-11-05T02:30:00.000Z
4,2023,11,regular,2023-11-08T00:00:00.000Z,2023-11-12T20:00:00.000Z
5,2023,12,regular,2023-11-15T00:00:00.000Z,2023-11-19T03:30:00.000Z
6,2023,13,regular,2023-11-21T05:00:00.000Z,2023-11-26T04:00:00.000Z
7,2023,15,regular,2023-12-09T20:00:00.000Z,2023-12-09T20:00:00.000Z
8,2023,11,postseason,2023-11-11T17:00:00.000Z,2023-11-11T21:00:00.000Z
9,2023,12,postseason,2023-11-18T18:00:00.000Z,2023-11-18T18:00:00.000Z


In [5]:
plays_api = cfbd.PlaysApi(api_config)
con = duckdb.connect('cfb.db')
for i in range(0,len(cal_df)):
    week = int(cal_df.loc[i,'week'])
    season_type = cal_df.loc[i,'season_type']
    season = int(cal_df.loc[i,'season'])
    start = time.time()
    print(f"Getting Season: {season}, Week: {week}, Season_Type: {season_type}, start_time: {start}\n")
    plays = plays_api.get_plays(year=season,week=week,season_type=season_type)
    print(f"Got plays\n")
    week_df = pd.DataFrame().from_records([pbp.to_dict() for pbp in plays])
    if season_type == 'regular' and week == 1:
        result = con.execute(f"CREATE TABLE plays_{season} AS SELECT * FROM week_df")
    else:
        result = con.execute(f"INSERT INTO plays_{season} SELECT * FROM week_df")
    end = time.time()
    print(f"Loaded into duckdb. Result: {result}, end_time: {end}, elapsed: {end - start}\n")

Getting Season: 2023, Week: 7, Season_Type: regular, start_time: 1699912239.4924395

Got plays

Loaded into duckdb. Result: <duckdb.duckdb.DuckDBPyConnection object at 0x7ff4d5fbaef0>, end_time: 1699912257.315463, elapsed: 17.823023557662964

Getting Season: 2023, Week: 8, Season_Type: regular, start_time: 1699912257.3156748

Got plays

Loaded into duckdb. Result: <duckdb.duckdb.DuckDBPyConnection object at 0x7ff4d5fbaef0>, end_time: 1699912302.8597553, elapsed: 45.54408049583435

Getting Season: 2023, Week: 9, Season_Type: regular, start_time: 1699912302.8599718

Got plays

Loaded into duckdb. Result: <duckdb.duckdb.DuckDBPyConnection object at 0x7ff4d5fbaef0>, end_time: 1699912375.4114852, elapsed: 72.55151343345642

Getting Season: 2023, Week: 10, Season_Type: regular, start_time: 1699912375.4116573

Got plays

Loaded into duckdb. Result: <duckdb.duckdb.DuckDBPyConnection object at 0x7ff4d5fbaef0>, end_time: 1699912490.682653, elapsed: 115.27099561691284

Getting Season: 2023, Week:

InvalidInputException: Invalid Input Error: Need a DataFrame with at least one column

In [4]:
%%jupysql
SELECT 
    games.start_date,
    games.Season,
    games.Week,
    games.Season_type
FROM plays
inner join games on plays.game_id = games.id
ORDER BY season desc, start_date desc
LIMIT 100

,start_date,season,week,season_type
0,2018-01-09T01:00:00.000Z,2017,1,postseason
1,2018-01-09T01:00:00.000Z,2017,1,postseason
2,2018-01-09T01:00:00.000Z,2017,1,postseason
3,2018-01-09T01:00:00.000Z,2017,1,postseason
4,2018-01-09T01:00:00.000Z,2017,1,postseason
...,...,...,...,...
95,2018-01-09T01:00:00.000Z,2017,1,postseason
96,2018-01-09T01:00:00.000Z,2017,1,postseason
97,2018-01-09T01:00:00.000Z,2017,1,postseason
98,2018-01-09T01:00:00.000Z,2017,1,postseason


In [9]:
pbp_df.columns

Index(['season', 'week', 'season_type'], dtype='object')